# 📊 **Pipeline de Processamento de Linguagem Natural (NLP)**

## **Do Texto Bruto à Modelagem Preditiva**

Neste projeto aplico técnicas clássicas de Processamento de Linguagem Natural (NLP)
para transformar textos em representações numéricas e aplicar modelos de Machine Learning.

O objetivo é demonstrar domínio prático do pipeline completo e análise de texto com técnicas de NLP.

Cada etapa está organizada com descrição conceitual e explicação do que está sendo realizado.



---

## 🔎 **Pipeline do Projeto**

Este notebook segue as seguintes etapas:

1. **Preparação dos dados** – `pandas`, `numpy` e carregamento da base.  

2. **Pré-processamento textual** – `regex`, `nltk` (stopwords), padronização para lowercase e limpeza de caracteres.  

3. **Vetorização** – `CountVectorizer` (Bag of Words) e `TfidfVectorizer` (TF-IDF) para transformação do texto em matriz esparsa.  

4. **Similaridade** – Similaridade do cosseno (`cosine similarity`) para medir proximidade entre documentos.  

5. **Modelagem** – `LogisticRegression` (scikit-learn) para classificação supervisionada.  

6. **Avaliação** – `classification_report` e `confusion_matrix` para análise de desempenho.

---

# **Importação das Bibliotecas** 🧰

In [1]:
# 📊 Manipulação de dados
import pandas as pd
import numpy as np

# 🧹 Processamento de texto
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# 🔢 Vetorização (Representação Numérica)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# 🧠 Modelos Baseados em Transformers
from transformers import pipeline

# 📐 Similaridade entre documentos
from sklearn.metrics.pairwise import cosine_similarity

# 🧠 Topic Modeling
from sklearn.decomposition import LatentDirichletAllocation

# 🤖 Modelagem e Avaliação (Machine Learning Clássico)
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## 🔹 **Etapa 1 – Coleta e Carregamento dos Dados** 💾

Nesta seção realizamos a clonagem do repositório público contendo o dataset
de reviews de e-commerce.

Os dados utilizados são **avaliações de produtos do Mercado Livre**,
armazenadas em formato JSON.

O código abaixo:

- Clona o repositório via GitHub
- Lista os arquivos disponíveis
- Carrega os arquivos JSON
- Concatena os dados em um único DataFrame para análise

In [2]:
!git clone https://github.com/octaprice/ecommerce-product-dataset.git
!ls ecommerce-product-dataset/data/mercadolivre_com_br


df1 = pd.read_json("ecommerce-product-dataset/data/mercadolivre_com_br/reviews_mercadolivre_com_br_1.json")
df2 = pd.read_json("ecommerce-product-dataset/data/mercadolivre_com_br/reviews_mercadolivre_com_br_2.json")
df = pd.concat([df1, df2], ignore_index=True)

Cloning into 'ecommerce-product-dataset'...
remote: Enumerating objects: 42, done.
remote: Total 42 (delta 0), reused 0 (delta 0), pack-reused 42 (from 1)
Receiving objects: 100% (42/42), 39.14 MiB | 10.91 MiB/s, done.
Resolving deltas: 100% (7/7), done.
Updating files: 100% (8/8), done.
reviews_mercadolivre_com_br_1.json  reviews_mercadolivre_com_br_2.json


## 🔹 **Etapa 2 – Modelos de Emoção e Análise de Sentimento** 😊🎭

Nesta etapa utilizamos modelos baseados em **Transformers** capaz de capturar melhor o **contexto das frases** para analisar textos
sob duas perspectivas:

- 🎭 **Análise de Emoção** → identifica emoções específicas **(alegria, tristeza, raiva, medo, surpresa, nojo)**.
- 😊 **Análise de Sentimento** → classifica a polaridade geral **(positivo, neutro, negativo)**.

🤖 **Carregamento do Modelo de Emoção** 🎭

In [3]:
classificador_emocao = pipeline(
    task="text-classification",
    model="bhadresh-savani/distilbert-base-uncased-emotion",
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/104 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

🔎 **Função para Detecção de Emoção**

Recebe um texto e retorna a emoção predominante identificada pelo modelo pré-treinado.

In [4]:
def detectar_emocao(texto):

    texto = str(texto)[:512]
    resultado = classificador_emocao(texto)[0]

    return resultado['label']

📊 **Aplicação do Modelo em uma Amostra do Dataset**

In [5]:
# Seleciona amostra para demonstração
df_demo = df.sample(200, random_state=42).copy()

# Executa inferência em lote
resultados = classificador_emocao(
    df_demo["content"].astype(str).tolist()
)

# Armazena emoções previstas
df_demo["emocao"] = [r["label"] for r in resultados]

df_demo.head()

,date,rating,content,product_url,emocao
170425,11 abr. 2024,5,Bom.,https://www.mercadolivre.com.br/wella-professi...,anger
84922,14 jan. 2024,5,Boa.,https://www.mercadolivre.com.br/premier-paste-...,anger
52542,22 jul. 2024,4,É muito cheiroso.\nAgora é só usar pra vê se p...,https://www.mercadolivre.com.br/kit-completo-5...,joy
126882,14 fev. 2025,5,Oti.,https://www.mercadolivre.com.br/kit-verniz-sha...,anger
154949,21 jul. 2024,5,Amei. Original. Meu cabelo ficou lindo.,https://www.mercadolivre.com.br/lanza-healing-...,joy


🤖 **Carregamento do Modelo de Sentimento** 😊

In [6]:
classificador_sentimento = pipeline(
    task="sentiment-analysis",
    model="cardiffnlp/twitter-xlm-roberta-base-sentiment"
)

def detectar_sentimento(texto):
    texto = str(texto)[:512]
    resultado = classificador_sentimento(texto)[0]
    return resultado['label']

config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

XLMRobertaForSequenceClassification LOAD REPORT from: cardiffnlp/twitter-xlm-roberta-base-sentiment
Key                             | Status     |  | 
--------------------------------+------------+--+-
roberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

📊 **Aplicação do Modelo na Mesma Amostra**

In [7]:
resultados_sentimento = classificador_sentimento(
    df_demo["content"].astype(str).tolist()
)

df_demo["sentimento"] = [r["label"] for r in resultados_sentimento]

**👀 Visualização e Análise das Emoções**

In [8]:
df_demo[['content','emocao','sentimento']].sample(5)

,content,emocao,sentimento
48504,Bom produto. Parece que cumpre cumpre o que pr...,anger,positive
198469,"Eu já adoro esta linha , sou apaixonada no che...",joy,positive
204969,Muito bom.,anger,positive
73890,Ótimo.,anger,positive
125097,"Produto vazou metade do produto na caixa, caix...",joy,negative


In [9]:
df_demo['sentimento'].value_counts().reset_index()

,sentimento,count
0,positive,165
1,negative,27
2,neutral,8


# 🔹 **Etapa 3 – Detecção de Inconsistências** ⚠️



### 📊 **Objetivo**
Identificar contradições entre a **nota (rating)** e o **sentimento previsto no texto**.

Consideramos inconsistência quando:

- ⭐ Nota alta (4 ou 5) → sentimento negativo  
- ⭐ Nota baixa (1 ou 2) → sentimento positivo  

Essa análise ajuda a identificar possíveis erros de rotulagem,
ironia textual ou limitações do modelo.

🔎 **Definição das regras de inconsistência**

In [10]:
inconsistencias = df_demo[
    (
        (df_demo["rating"] >= 4) & (df_demo["sentimento"] == "negative")
    )
    |
    (
        (df_demo["rating"] <= 2) & (df_demo["sentimento"] == "positive")
    )
]

# Visualizar exemplos
inconsistencias[["content", "rating", "sentimento"]].head()

,content,rating,sentimento
126882,Oti.,5,negative
22571,,5,negative
188619,Apresentei hipersensibilidade tardia com sinto...,5,negative
34057,Diminuiu a caspa e a coceira na terceira lavag...,4,negative
23208,Atentem apenas que há dois tamanhos desse prot...,5,negative


In [11]:
print('Inconsistentes:', len(inconsistencias))
print('%:', round(len(inconsistencias)/len(df_demo)*100, 2))

Inconsistentes: 13
%: 6.5


### 📌 **Interpretação**

A presença de inconsistências pode indicar:

- Uso irônico da linguagem
- Ambiguidade textual
- Diferença entre percepção emocional e avaliação final
- Limitações do modelo pré-treinado

Essa etapa adiciona uma camada analítica ao projeto,
indo além da simples classificação automática.

**Reviews muito curtas apresentaram maior proporção de inconsistências, sugerindo que a baixa quantidade de contexto textual pode dificultar a interpretação correta pelo modelo.**

# 🔹 **Etapa 4 – Detecção de Temas (Topic Modeling)** 🧩

### 📌 **Objetivo**
Identificar automaticamente os principais temas presentes nas reviews,
utilizando modelagem não supervisionada.

Aplicamos:
- **Bag of Words (CountVectorizer)** para transformar texto em matriz numérica  
- **LDA (Latent Dirichlet Allocation)** para descobrir grupos de palavras recorrentes  

Essa abordagem permite responder:

> “Quais são os principais assuntos discutidos nas avaliações?”

### 🔢 **Vetorização (Bag of Words)**

In [12]:
# Stopwords em português
nltk.download("stopwords")
stopwords_pt = stopwords.words("portuguese")

# Vetorização do texto
vetorizador = CountVectorizer(
    stop_words=stopwords_pt,
    max_features=1000
)

matriz_texto = vetorizador.fit_transform(df_demo["content"])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### **🧠 Treinamento do Modelo LDA**

In [13]:
# Treinamento do modelo de tópicos
modelo_lda = LatentDirichletAllocation(
    n_components=5,
    random_state=42
)

modelo_lda.fit(matriz_texto)

LatentDirichletAllocation(n_components=5, random_state=42)

### 🔎 **Principais Palavras por Tema**

In [14]:
# Visualizar palavras mais relevantes de cada tema
palavras = vetorizador.get_feature_names_out()

for indice_tema, tema in enumerate(modelo_lda.components_):
    principais_palavras = [palavras[i] for i in tema.argsort()[-10:]]
    print(f"Tema {indice_tema+1}")
    print(principais_palavras)

Tema 1
['adorei', 'tempo', 'ótima', 'cumpre', 'cabelo', 'maravilhoso', 'recomendo', 'uso', 'produto', 'bom']
Tema 2
['realmente', 'cabelos', 'cabelo', 'caixa', 'faz', 'shampoo', 'melhor', 'excelente', 'produto', 'maravilhoso']
Tema 3
['lindo', 'bom', 'comprei', 'super', 'veio', 'amo', 'produto', 'cabelo', 'ficou', 'comprar']
Tema 4
['ainda', 'bem', 'excelente', 'bom', 'amei', 'shampoo', 'ótimo', 'produto', 'gostei', 'cabelo']
Tema 5
['maravilhoso', 'macio', 'cabelos', 'deixa', 'super', 'cheiroso', 'produto', 'perfeito', 'ótimo', 'cabelo']


### 📊 **Análise dos Temas (Amostra)**

Identificamos o **tema dominante** de cada review com base na maior
probabilidade atribuída pelo LDA.

Em seguida, analisamos a distribuição dos temas
e sua relação com o sentimento.

In [15]:
# 4. Identificar tema principal de cada review
matriz_temas = modelo_lda.transform(matriz_texto)

df_demo["tema_principal"] = matriz_temas.argmax(axis=1)

**Análise Estatística Descritiva dos temas**

In [16]:
df_demo["tema_principal"].value_counts().reset_index()

,tema_principal,count
0,0,53
1,4,50
2,3,45
3,1,31
4,2,21


In [17]:
df_demo.groupby('tema_principal')['rating'].mean().reset_index()

,tema_principal,rating
0,0,4.698113
1,1,4.741935
2,2,4.571429
3,3,4.711111
4,4,4.600000


### 😊 **Distribuição de Sentimento por Tema**

Analisamos como o sentimento se distribui dentro de cada tema,
permitindo identificar quais assuntos estão mais associados
a avaliações positivas ou negativas.

In [18]:
df_demo.groupby(["tema_principal", "sentimento"]).size().unstack().reset_index()

sentimento,tema_principal,negative,neutral,positive
0,0,7,3,43
1,1,4,1,26
2,2,5,2,14
3,3,5,1,39
4,4,6,1,43


## 🔹 **Etapa 5 – Análise de Similaridade Textual** 📐

**🎯 Objetivo**

Nesta etapa utilizamos **TF-IDF** para representar os textos
e aplicamos **similaridade do cosseno** para medir proximidade entre reviews.

Essa abordagem permite identificar textos semanticamente semelhantes,
servindo como base para sistemas de recomendação ou busca textual.

🔢 **Representação vetorial com TF-IDF**


In [19]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords_pt = stopwords.words("portuguese")


tfidf_vectorizer = TfidfVectorizer(
    stop_words=stopwords_pt,
    min_df=5
)

X_tfidf = tfidf_vectorizer.fit_transform(df["content"])

📚 **Vocabulário criado pelo modelo**

In [20]:
vocab_tfidf = tfidf_vectorizer.get_feature_names_out()

📐 **Similaridade do cosseno entre duas reviews**


In [21]:
i, j = 11, 20

print("Review A:\n", df.loc[i, "content"])
print("\nReview B:\n", df.loc[j, "content"])

similaridade = cosine_similarity(
    X_tfidf[i],
    X_tfidf[j]
)[0][0]

print("\nSimilaridade:", similaridade)

Review A:
 Tudo de bom! gostei! eu recomendo.

Review B:
 Gostei bastante idrata da brilho! o cheiro da idratação e meio forte lembra uma progressiva! mas estou gostando do resultado pretendo comprar mais produto da linha.

Similaridade: 0.07180956523736445


🔎 **Encontrar textos semelhantes a uma review específica**

In [22]:
idx_base = 11

# Calcular similaridade com todas as reviews
similaridades = cosine_similarity(
    X_tfidf[idx_base],
    X_tfidf
)[0]

# Ordenar por maior similaridade
df_sim = (
    df.assign(similaridade=similaridades)
      .sort_values("similaridade", ascending=False)
)

# Top 5 mais semelhantes (excluindo a própria)
df_sim.iloc[1:6][["rating", "similaridade", "content"]]

,rating,similaridade,content
94666,5,0.877802,Tudo de bom! recomendo 😊.
6802,5,0.877802,Tudo de bom. Recomendo.
13941,5,0.830025,Esse produto é tudo de bom.\nEu recomendo.
50083,5,0.780823,"Tudo de bom, amei. Recomendo."
51179,4,0.769854,Não é tudo aquilo no meu cabelo não gostei.


🔍 **Busca semântica a partir de um texto digitado**

In [23]:
consulta = "produto deixou meu cabelo macio e cheiroso"

vetor_consulta = tfidf_vectorizer.transform([consulta])

similaridades_consulta = cosine_similarity(
    vetor_consulta,
    X_tfidf
)[0]

df_busca = (
    df.assign(similaridade=similaridades_consulta)
      .sort_values("similaridade", ascending=False)
)

df_busca.head(5)[["rating", "similaridade", "content"]]

,rating,similaridade,content
53432,5,1.000000,Produto cheiroso e deixou meu cabelo macio.
204120,5,0.963071,Muito cheiroso e deixou meu cabelo macio.
156180,5,0.963071,Deixou meu cabelo mais macio e muito cheiroso.
142836,5,0.963071,É muito cheiroso e deixou o meu cabelo muito m...
5455,5,0.946907,Produto maravilhoso deixou meu cabelo macio e ...


# 🔹 **Etapa 6 – Classificação de Texto com Regressão Logística** 🤖

Nesta etapa treinamos um modelo supervisionado para classificar reviews
em **negativo, neutro ou positivo**, a partir da nota atribuída.

**Essa abordagem é amplamente utilizada em:**
- Classificação de reviews
- Detecção de spam
- Categorização de tickets
- Identificação de intenção em chatbots

### 🎯 **Criação da Variável Alvo**

Para transformar o problema em uma tarefa de classificação supervisionada,
convertimos a nota numérica (`rating`) em categorias de sentimento:

- 1 ou 2 → **negativo**
- 3 → **neutro**
- 4 ou 5 → **positivo**

**Aqui definimos a classe que o modelo irá prever, baseada na nota atribuída pelo usuário.**

In [24]:
def classificar_rating(r):
    if r <= 2:
        return "negativo"
    elif r == 3:
        return "neutro"
    else:
        return "positivo"

df["classe_sentimento"] = df["rating"].apply(classificar_rating)

### 🔎 **Definição das variáveis**

Aqui separamos o que será usado como entrada **(texto)**
e o que o modelo deve prever **(classe de sentimento)**.

In [25]:
X = df["content"].astype(str)
y = df["classe_sentimento"]

### ✂️ **Divisão em Treino e Teste**

Separamos os dados em **80% treino** e **20% teste**.

O objetivo é treinar o modelo em uma parte dos dados
e avaliar seu desempenho em dados não vistos garantindo uma avaliação mais confiável.


In [26]:
from sklearn.model_selection import train_test_split

X_treino, X_teste, y_treino, y_teste = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

### 🔤 **Vetorização com TF-IDF**

- Transformação do texto em representação numérica.

- max_features=5000 limita o vocabulário

- O modelo passa a trabalhar com vetores numéricos

In [27]:
vetorizador = TfidfVectorizer(
    max_features=5000,
    stop_words=None
)

X_treino_vet = vetorizador.fit_transform(X_treino)
X_teste_vet = vetorizador.transform(X_teste)

### 📈 **Treinamento do Modelo**

Treinamos uma Regressão Logística com **balanceamento de classes** para lidar com desbalanceamento do dataset.

In [28]:
modelo = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    random_state=42
)

modelo.fit(X_treino_vet, y_treino)

LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)

### 🔮**Realização das Previsões**
Aplicamos o modelo no conjunto de teste.

**A variável predicoes** foi utilizada na etapa de avaliação do modelo, sendo essencial para o **cálculo das métricas de desempenho**, como **accuracy** e **relatório de classificação**, podendo ainda ser explorada em análises adicionais de erro.

In [29]:
predicoes = modelo.predict(X_teste_vet)

### 📊**Avaliação do Modelo**

Avaliamos o desempenho utilizando:

- Accuracy
- Precision
- Recall
- F1-score

In [30]:
print("Accuracy:", accuracy_score(y_teste, predicoes))
print("\nRelatório de classificação:")
print(classification_report(y_teste, predicoes))

Accuracy: 0.8605735546374815

Relatório de classificação:
              precision    recall  f1-score   support

    negativo       0.51      0.72      0.60      1999
      neutro       0.13      0.55      0.22      1143
    positivo       0.99      0.88      0.93     38249

    accuracy                           0.86     41391
   macro avg       0.55      0.72      0.58     41391
weighted avg       0.94      0.86      0.89     41391



### 📊 **Interpretação dos Resultados**

O modelo alcançou **86% de acurácia**, com desempenho elevado na classe **positiva**, que concentra a maior parte das amostras.

As classes **negativa** e principalmente **neutra** apresentaram métricas inferiores, refletindo o desbalanceamento do dataset.

Foi utilizado `class_weight="balanced"` na Regressão Logística para tentar reduzir esse efeito, ajustando o peso das classes minoritárias durante o treinamento. Ainda assim, observa-se que o desbalanceamento impacta o desempenho final.

### 🧪**Teste com Novos Textos**

Aplicação prática do modelo em exemplos manuais.

In [31]:
# Exemplo 1
texto = ['Gostei muito, indico.']
texto_vet = vetorizador.transform(texto)
print("Predição:", modelo.predict(texto_vet)[0])

# Exemplo 2
texto = ['Até que é bom.']
texto_vet = vetorizador.transform(texto)
print("Predição:", modelo.predict(texto_vet)[0])

# Exemplo 3
texto = ['Péssimo produto, odiei o cheiro.']
texto_vet = vetorizador.transform(texto)
print("Predição:", modelo.predict(texto_vet)[0])

Predição: positivo
Predição: neutro
Predição: negativo


# **📌 Conclusão**

Neste projeto aplicamos um pipeline completo de NLP, incluindo:

- Pré-processamento textual  
- Representação vetorial (BoW e TF-IDF)  
- Modelagem supervisionada  
- Topic Modeling (LDA)  
- Similaridade textual  
- Análise com Transformers  

O teste com novos textos demonstra a aplicação prática do modelo,
mostrando sua capacidade de generalizar para avaliações não vistas.

Como próximos passos, seria possível:
- Ajustar hiperparâmetros  
- Aplicar técnicas de balanceamento mais avançadas  
- Comparar com modelos baseados em Transformers  

**O projeto evidencia a aplicação integrada de técnicas clássicas e modernas de NLP em um cenário real de reviews de e-commerce.**